In [29]:
!pip install --upgrade pip --user

Looking in indexes: http://10.209.156.240:8081/repository/python-all/simple
Requirement already up-to-date: pip in /home/jovyan/.local/lib/python3.7/site-packages (20.1.1)


In [30]:
!pip install exifread --user
!pip install whatimage --user
!pip install pyheif --user

Looking in indexes: http://10.209.156.240:8081/repository/python-all/simple
Looking in indexes: http://10.209.156.240:8081/repository/python-all/simple
Looking in indexes: http://10.209.156.240:8081/repository/python-all/simple


# Tokenizing and Embedding All Trash Data 
- Convert images into .npy files
- Read excels and converts labels and metadata into .npy, .csv files.

In [1]:
import os
import io
import ast
import json
import torch
import torch.nn as nn
import torchvision
import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image
import PIL
import pyheif
import exifread

In [3]:
ROOT = Path('../asun/Smart-Trash/data/isbnet')
print(os.listdir(ROOT))

['isbnet_pure_index.csv', 'cans', 'embedded', 'Trash Category Metadata.xlsx', 'metadata.csv', 'paper', 'landfill', 'Trash Can Location Descriptors.xlsx', 'plastic', 'tetrapak', 'isbnet_index.csv', '.ipynb_checkpoints', 'metalabels.json']


In [4]:
# Get all the directories of each categories and remove .ipynb_checkpoints from directories
categories = [ROOT / v for v in os.listdir(ROOT) if os.path.isdir(ROOT / v)][:-1]

In [5]:
trashcans = []
for c in categories:
    for t in os.listdir(c):
        if t[-4:] != '.jpg' and t != '.DS_Store':
            trashcans.append(c / t)

In [5]:
trashcans

[PosixPath('../asun/Smart-Trash/data/isbnet/cans/9A'),
 PosixPath('../asun/Smart-Trash/data/isbnet/cans/6B'),
 PosixPath('../asun/Smart-Trash/data/isbnet/cans/5A'),
 PosixPath('../asun/Smart-Trash/data/isbnet/cans/5B'),
 PosixPath('../asun/Smart-Trash/data/isbnet/cans/8B'),
 PosixPath('../asun/Smart-Trash/data/isbnet/cans/1B'),
 PosixPath('../asun/Smart-Trash/data/isbnet/cans/3B'),
 PosixPath('../asun/Smart-Trash/data/isbnet/cans/12A'),
 PosixPath('../asun/Smart-Trash/data/isbnet/cans/1A'),
 PosixPath('../asun/Smart-Trash/data/isbnet/cans/4B'),
 PosixPath('../asun/Smart-Trash/data/isbnet/cans/10A'),
 PosixPath('../asun/Smart-Trash/data/isbnet/cans/7B'),
 PosixPath('../asun/Smart-Trash/data/isbnet/cans/11A'),
 PosixPath('../asun/Smart-Trash/data/isbnet/cans/2B'),
 PosixPath('../asun/Smart-Trash/data/isbnet/cans/3A'),
 PosixPath('../asun/Smart-Trash/data/isbnet/cans/4A'),
 PosixPath('../asun/Smart-Trash/data/isbnet/cans/2A'),
 PosixPath('../asun/Smart-Trash/data/isbnet/cans/6A'),
 PosixP

## Read Meta-Labels

In [6]:
location = pd.read_excel(ROOT / 'Trash Can Location Descriptors.xlsx').fillna(0)
time_meta = pd.read_excel(ROOT / 'Trash Category Metadata.xlsx').fillna(0)

In [7]:
location

,Trash Can ID,Landmark Descriptors,"Point-Wise Distance (Pixels) (1st, 2nd; 8.5px/m, 8.6px/m)","Point-Wise Distance (Meters) (1st, 2nd; 8.5px/m, 8.6px/m)","Max Probability Time (8, 9, 0, 1, 2, 3, 4, 5, 6, 7)","Max Probability Time (8, 9, 0, 1, 2, 3, 4, 5, 6, 7)",Single-Floor Distance
0,1A,"entrance_exit, bathroom, couch_area, stairwell...","(20, 108), (13, 57), (108, 37), (139, 21), (12...","(2.353, 12.706), (1.529, 6.706), (12.706, 4.35...","(1, 0, 0, 1, 1, 0, 0, 1, 1, 1)","[0.3181, 0.3995, 0.4035, 0.3404, 0.1372, 0.373...","(12.922), (6.878), (13.431), (16.539), (23.289)"
1,2A,"entrance_exit, bathroom, couch_area, stairwell...","(17, 101), (0, 62), (101, 40), (127, 18), (112...","(2.0, 11.882), (0.0, 7.294), (11.882, 4.706), ...","(1, 0, 0, 1, 1, 0, 0, 1, 1, 1)","[0.28, 0.3558, 0.4557, 0.4071, 0.1449, 0.3894,...","(12.049), (7.249), (12.780), (15.090), (22.307..."
2,3A,"entrance_exit, bathroom, couch_area, stairwell","(14, 107), (8, 57), (104, 36), (134, 21)","(1.647, 12.588), (0.941, 6.706), (12.235, 4.23...","(1, 0, 0, 0, 0, 0, 0, 1, 1, 1)","[0.2473, 0.3106, 0.3137, 0.2647, 0.29, 0.29, 0...","(12.695), (6.772), (12.947), (15.957)"
3,4A,"bathroom, couch_area, stairwell",0,0,0,0,0
4,8A,"cafeteria, stairwell, bathroom, library, print...","(151, 275), (65, 16), (79, 119), (252, 123), (...","(17.558, 31.977), (7.558, 1.86), (9.186, 13.83...","(0, 0, 0, 1, 1, 1, 1, 1, 0, 0)","[0.3081, 0.312, 0.3996, 0.4481, 0.1613, 0.4335...","(36.480), (7.784), (16.509), (32.606), (41.311..."
5,9A,"theater, printer, stairwell, bathroom, library","(207, 67), (19, 68), (72, 52), (60, 154), (236...","(24.07, 7.791), (2.209, 7.907), (8.372, 6.047)...","(0, 0, 0, 0, 0, 0, 0, 1, 1, 1)","[0.3068, 0.3068, 0.3068, 0.3068, 0.1129, 0.505...","(25.299), (8.210), (10.327), (19.218), (28.100)"
6,10A,"entrance_exit, theater, gym, pool, printer","(34, 48), (128, 152), (149, 158), (140, 525), ...","(3.953, 5.581), (14.884, 17.674), (17.326, 18....","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","[0.2571, 0.3227, 0.3227, 0.2571, 0.0935, 0.424...","(6.839), (23.106), (25.253), (63.180), (35.012)"
7,11A,"entrance_exit, theater, gym, stairwell","(7, 207), (140, 12), (139, 317), (36, 175)","(0.814, 24.07), (16.279, 1.395), (16.163, 36.8...","(1, 1, 0, 1, 1, 0, 1, 0, 0, 0)","[0.2185, 0.2743, 0.2743, 0.2185, 0.2161, 0.360...","(24.084), (16.339), (40.248), (20.775)"
8,12A,"library, printer, cafeteria","(72, 112), (31, 3), (468, 286)","(8.372, 13.023), (3.605, 0.349), (54.419, 33.256)","(0, 0, 1, 1, 1, 1, 1, 1, 0, 0)","[0.3528, 0.353, 0.3566, 0.3776, 0.1539, 0.4185...","(15.482), (3.622), (63.776)"
9,1B,"bathroom, couch_area, stairwell, printer, lounge","(1, 29), (92, 74), (112, 21), (78, 36), (62, 63)","(0.116, 3.372), (10.698, 8.605), (13.023, 2.44...","(0, 1, 1, 1, 1, 1, 1, 1, 1, 0)","[0.3011, 0.3049, 0.3905, 0.4379, 0.1576, 0.423...","(3.374), (13.729), (13.250), (9.990), (10.278)"


In [8]:
time_meta.iloc[:, 0]

0    Landfill
1        Cans
2     Plastic
3       Paper
4    Tetrapak
Name: Category , dtype: object

The general format for the path will be:
`../<username>/Smart-Trash/data/isbnet/<category>/<trashcan ID>/`. We can leverage this to create meta data.

**DataFrame Columns:**
- filpath (`str`)
- label/category (`str`)
- category time metadata (`Tuple<float>`)
- trashcan time metadata (`Tuple<float>`)
- trashcan location metadata (`Tuple<float>`)
- location descriptors (`List<str>`)

Some of the files in the mix are HEICs so we have to convert them back into .jpgs

In [9]:
def read_heic(path: Path):
    heif_file = pyheif.read(path)
    image = Image.frombytes(
        heif_file.mode,
        heif_file.size,
        heif_file.data,
        'raw',
        heif_file.mode,
        heif_file.stride
    )
    image.save(path, 'jpeg')

In [10]:
df = []
for trashcan in trashcans:
    for filename in os.listdir(trashcan):
        if filename == '.DS_Store':
            continue
        if filename == '.ipynb_checkpoints':
            continue
        if filename[-5:] == '..jpg':
            os.rename(trashcan / filename, trashcan / f'{filename[:-5]}.jpg')
            filename = f'{filename[:-5]}.jpg'
        try: 
            Image.open(trashcan / filename)
        except OSError:
            read_heic(trashcan / filename)
        pathlist = str(trashcan).split('/')
        category, trashcan_id = pathlist[-2], pathlist[-1]
        pathlist = trashcan / filename
        category_time = time_meta[time_meta.iloc[:,0]==category.title()].iloc[0, 1]
        trashcan_time = location[location.iloc[:, 0]==trashcan_id].iloc[0, 4]
        trashcan_location = location[location.iloc[:, 0]==trashcan_id].iloc[0, 3]
        trashcan_location = f'[{trashcan_location.strip()}]'
        landmarks = location[location.iloc[:, 0]==trashcan_id].iloc[0, 1].split(', ')
        df.append([pathlist, category.lower(), category_time.strip(), trashcan_id,
                   trashcan_time.strip(), trashcan_location, landmarks])

In [11]:
metadata = pd.DataFrame(df, columns=['filepath', 'category', 'categorial_time',
                                     'trashcan_id', 'trashcan_time', 
                                     'trashcan_location', 'landmarks'])

In [12]:
metadata.to_csv(ROOT / 'metadata.csv', index=False)

In [13]:
metadata

,filepath,category,categorial_time,trashcan_id,trashcan_time,trashcan_location,landmarks
0,../asun/Smart-Trash/data/isbnet/cans/9A/202001...,cans,"(0, 0, 0, 1, 1, 1, 0, 0, 0, 0)",9A,"(0, 0, 0, 0, 0, 0, 0, 1, 1, 1)","[(24.07, 7.791), (2.209, 7.907), (8.372, 6.047...","[theater, printer, stairwell, bathroom, library]"
1,../asun/Smart-Trash/data/isbnet/cans/9A/202001...,cans,"(0, 0, 0, 1, 1, 1, 0, 0, 0, 0)",9A,"(0, 0, 0, 0, 0, 0, 0, 1, 1, 1)","[(24.07, 7.791), (2.209, 7.907), (8.372, 6.047...","[theater, printer, stairwell, bathroom, library]"
2,../asun/Smart-Trash/data/isbnet/cans/9A/202001...,cans,"(0, 0, 0, 1, 1, 1, 0, 0, 0, 0)",9A,"(0, 0, 0, 0, 0, 0, 0, 1, 1, 1)","[(24.07, 7.791), (2.209, 7.907), (8.372, 6.047...","[theater, printer, stairwell, bathroom, library]"
3,../asun/Smart-Trash/data/isbnet/cans/6B/cans29...,cans,"(0, 0, 0, 1, 1, 1, 0, 0, 0, 0)",6B,"(0, 1, 0, 1, 0, 0, 1, 0, 0, 0)","[(0.465, 7.442), (15.116, 6.744), (10.349, 7.5...","[bathroom, stairwell, couch_area]"
4,../asun/Smart-Trash/data/isbnet/cans/6B/IMG_73...,cans,"(0, 0, 0, 1, 1, 1, 0, 0, 0, 0)",6B,"(0, 1, 0, 1, 0, 0, 1, 0, 0, 0)","[(0.465, 7.442), (15.116, 6.744), (10.349, 7.5...","[bathroom, stairwell, couch_area]"
...,...,...,...,...,...,...,...
883,../asun/Smart-Trash/data/isbnet/tetrapak/8A/tr...,tetrapak,"(0, 1, 1, 1, 1, 1, 0, 1, 1, 0)",8A,"(0, 0, 0, 1, 1, 1, 1, 1, 0, 0)","[(17.558, 31.977), (7.558, 1.86), (9.186, 13.8...","[cafeteria, stairwell, bathroom, library, prin..."
884,../asun/Smart-Trash/data/isbnet/tetrapak/8A/tr...,tetrapak,"(0, 1, 1, 1, 1, 1, 0, 1, 1, 0)",8A,"(0, 0, 0, 1, 1, 1, 1, 1, 0, 0)","[(17.558, 31.977), (7.558, 1.86), (9.186, 13.8...","[cafeteria, stairwell, bathroom, library, prin..."
885,../asun/Smart-Trash/data/isbnet/tetrapak/8A/20...,tetrapak,"(0, 1, 1, 1, 1, 1, 0, 1, 1, 0)",8A,"(0, 0, 0, 1, 1, 1, 1, 1, 0, 0)","[(17.558, 31.977), (7.558, 1.86), (9.186, 13.8...","[cafeteria, stairwell, bathroom, library, prin..."
886,../asun/Smart-Trash/data/isbnet/tetrapak/8A/tr...,tetrapak,"(0, 1, 1, 1, 1, 1, 0, 1, 1, 0)",8A,"(0, 0, 0, 1, 1, 1, 1, 1, 0, 0)","[(17.558, 31.977), (7.558, 1.86), (9.186, 13.8...","[cafeteria, stairwell, bathroom, library, prin..."


Generate .json to help dataset creation. `meta` of the metadata.

In [47]:
metalabels = {'landmarks': {}, 'trashcans': []}
landmark_count = 0
for row in range(len(metadata)):
    if metadata.iloc[row]['trashcan_id'] not in metalabels['trashcans']:
        metalabels['trashcans'].append(metadata.iloc[row]['trashcan_id'])
    for landmark in metadata.iloc[row]['landmarks']:
        if landmark not in metalabels['landmarks'].keys():
            metalabels['landmarks'][landmark] = landmark_count
            landmark_count += 1

In [48]:
json.dump(metalabels, open(ROOT / 'metalabels.json', 'w+'))